In [34]:
from os import path
import pandas as pd
import numpy as np
import json
from openai import OpenAI
from tqdm import tqdm
from dotenv import load_dotenv
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from sentence_transformers import SentenceTransformer

from mint.config import DATA_DIR, RESULT_DIR

In [35]:
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [36]:
# CONFIG
ViABSA_BP_dir = path.join(DATA_DIR, 'ViABSA_BP')
test_file = path.join(ViABSA_BP_dir, 'data_test.csv')
test_df = pd.read_csv(test_file)

In [37]:
def transform_aspect_sentiment(df, start=0, end=None):
    aspects = [
        "stayingpower",
        "texture",
        "smell",
        "price",
        "others",
        "colour",
        "shipping",
        "packing"
    ]

    if end is None:
        end = len(df)

    result = []

    for idx, row in df.iloc[start:end].iterrows():
        entry = {
            "id": str(idx),
            "text": row['data'],
            "sentiments": []
        }

        for aspect in aspects:
            sentiment = row[f"{aspect}_label"]
            if sentiment == 1: 
                aspect_sentiment_value = row[aspect]
                if aspect_sentiment_value != 'none':
                    entry["sentiments"].append({
                        "aspect": aspect,
                        "sentiment": aspect_sentiment_value
                    })
                else:
                    entry["sentiments"].append({
                        "aspect": aspect,
                        "sentiment": "unknown"
                    })

        result.append(entry)

    return result

In [38]:
# SETUP DATA
aspects = ['stayingpower', 'texture', 'smell', 'price', 'others', 'colour', 'shipping', 'packing']
test_df[aspects] = test_df[aspects].fillna('none')

for aspect in aspects:
    test_df[aspect + '_label'] = (test_df[aspect] != 'none').astype(int)

test_json = transform_aspect_sentiment(test_df, 0, 100)
test_json[:5]

[{'id': '0',
  'text': 'Hàng đóng gói đẹp và chắc chắn, nhìn rất dễ thương và ưng bụng ạ!',
  'sentiments': [{'aspect': 'packing', 'sentiment': 'positive'}]},
 {'id': '1',
  'text': 'Cảm giác son bên trong rất là ít luôn í, đóng gói cẩn thận, dịch nhưng mà giao hàng khá nhanh',
  'sentiments': [{'aspect': 'shipping', 'sentiment': 'positive'},
   {'aspect': 'packing', 'sentiment': 'positive'}]},
 {'id': '2',
  'text': 'Son siêu đẹp luôn ý, mà y hình mà chụp có thể không giống lắm nhưng nhìn ngoài thì giống nha. Chất son mềm mướt nói chung là rất thíchhhh',
  'sentiments': [{'aspect': 'texture', 'sentiment': 'positive'}]},
 {'id': '3',
  'text': 'Siu đẹp luôn \r\nShop đóng gói cẩn thận lắm luôn \r\nLại thêm cả quà nữa\r\nNói chung là thích lắm',
  'sentiments': [{'aspect': 'packing', 'sentiment': 'positive'}]},
 {'id': '4',
  'text': 'Aúhihđcyihb gfxxth jj bhgfzđE G GHVHBTCEETXUBBIYCZRZRTCVUUBYRZXGVBIINVUTXZRCTIBONONBUXTZRTVKNNOUVTXEecyuvknibtcrztxuvbubibijvcytdgunonobiyvdtrdfyyghuojpmib

**Ý tưởng chọn example cho few-shot**
1.   Chọn ra những câu đa dạng về aspect từ dataset -> gôm cụm chúng lại để LLMs hiểu rõ hơn về pattern để học
2.   Chọn ra những câu khó, những câu phức tạp hơn 

Áp dụng K-Means Clustering + Chọn Centroid + Sampling Example khó

In [39]:
'''
    Nếu dùng TD-IDF thì sẽ chỉ chọn dựa vào tần suất từ, không hiểu nghĩa của câu -> bị trùng lặp
    Semantic grouping để chọn example đa dạng về nội dung, không chỉ về các từ
'''
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(test_df['data'].tolist(), show_progress_bar=True)

# KMeans clustering (70% đa dạng, 30% khó)
n_total = 10
n_diverse = int(n_total * 0.7)
n_hard = n_total - n_diverse

kmeans = KMeans(n_clusters=n_diverse, random_state=42, n_init=10)
kmeans.fit(embeddings)

# Lấy diverse example gần cluster centroid
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, embeddings)
diverse_examples = test_df.iloc[closest]

print(f"Selected {len(diverse_examples)} diverse examples:")
for i, row in diverse_examples.iterrows():
    print(f"\nExample {i}:\n{row['data']}")

Batches: 100%|██████████| 51/51 [00:08<00:00,  6.36it/s]


Selected 7 diverse examples:

Example 167:
Chất lượng sản phẩm rất tốt, phù hợp với giá tiền, chât son rất đẹp và thơm. Giao hàng nhanh, nên ủng hộ shop lâu dài . Rất thích sản phẩm ở đây

Example 213:
Nhận đc hàng nhưng giao hàng hơi lâu, mùi son thơm, mướt môi nhưng màu ko giống lắm

Example 1591:
đẹppppppppppp xĩuuuuuuuuuuuu hdjhsjsjsjdjdjjdjdjdjdjd

Example 340:
màu lên cũng bth ạ, đẹp. Giao hơi lâu thui ạ, mùi vẫn giống mấy khác ạ

Example 1619:
Son lên màu đẹp lắm nha,mùi thơm như kẹo socola á. Nói chung giá tiền rẻ nhưng chất lượng theo cảm nhận cá nhân mình thì tuyệt.mỗi tội mình chọn màu hơi tối khum hợp lắm huhu

Example 431:
shop giao hàng nhanh. Đóng gói cẩn thận. Chưa sử dụng nên chưa biết chất lượng. Sẽ tiếp tục ủng hộ shop

Example 1326:
Công dụng: son môi
Kết cấu: kem
Độ bền màu: Tương đối
Bôi lên đẹp lắm nhưng k lì lắm nha hhhhhhhh


In [40]:
all_indices = np.arange(len(test_df))

remaining_indices = list(set(all_indices) - set(closest))

# Chọn ngẫu nhiên n_hard example từ phần còn lại 
hard_examples = test_df.iloc[remaining_indices].sample(n=n_hard, random_state=42)

print(f"Selected {len(hard_examples)} hard examples:")
for i, row in hard_examples.iterrows():
    print(f"\nExample {i}:\n{row['data']}")

Selected 3 hard examples:

Example 135:
son sịn . màu đẹp jaisidiisisjsjsjsjdiididdiidjsjdjjdididi

Example 1432:
Shopee mall đóng gói hàng chắc chắn, màu son lên đẹp, dịch nên hàng từ 9/9 bh mới nhận

Example 833:
Son chất lượng nên mua nha mọi ngừoi rấ đáng giá ạ khá ok ạ nên ok ah


In [45]:
def select_few_shot_examples(df, text_column, n_total=10, model_name='all-MiniLM-L6-v2', random_state=42):
    n_diverse = int(n_total * 0.7)
    n_hard = n_total - n_diverse

    model = SentenceTransformer(model_name)
    embeddings = model.encode(df[text_column].tolist(), show_progress_bar=True)

    kmeans = KMeans(n_clusters=n_diverse, random_state=random_state, n_init=10)
    kmeans.fit(embeddings)

    closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, embeddings)
    diverse_df = df.iloc[closest]

    all_indices = np.arange(len(df))
    remaining_indices = list(set(all_indices) - set(closest))
    hard_df = df.iloc[remaining_indices].sample(n=n_hard, random_state=random_state)

    return diverse_df.reset_index(drop=True), hard_df.reset_index(drop=True)

# SET-UP FEW-SHOT EXAMPLES
diverse, hard = select_few_shot_examples(test_df, text_column='data', n_total=5)

few_shot_json =  transform_aspect_sentiment(diverse, 0, 100) + transform_aspect_sentiment(hard, 0, 100)


Batches: 100%|██████████| 51/51 [00:08<00:00,  6.32it/s]


In [42]:
gpt_functions = [
    {
        "name": "extract_aspect_sentiment",
        "description": "Extract aspects and sentiments from text",
        "parameters": {
            "type": "object",
            "properties": {
                "results": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "aspect": {
                                "type": "string",
                                "enum": aspects
                            },
                            "sentiment": {
                                "type": "string",
                                "enum": ["positive", "negative", "neutral"]
                            }
                        },
                        "required": ["aspect", "sentiment"]
                    }
                }
            },
            "required": ["results"]
        }
    }
]


In [43]:
def evaluate_aspect_sentiment(ground_truth, predictions):
    true_aspects = []
    pred_aspects = []

    true_aspect_sentiments = []
    pred_aspect_sentiments = []

    for gt_entry, pred_entry in zip(ground_truth, predictions):
        gt_sents = gt_entry['sentiments']
        gt_aspect_set = set()
        gt_aspect_sentiment_set = set()

        for item in gt_sents:
            gt_aspect_set.add(item['aspect'])
            gt_aspect_sentiment_set.add((item['aspect'], item['sentiment']))

        true_aspects.append(gt_aspect_set)
        true_aspect_sentiments.append(gt_aspect_sentiment_set)

        pred_sents = pred_entry['results']
        pred_aspect_set = set()
        pred_aspect_sentiment_set = set()

        for item in pred_sents:
            pred_aspect_set.add(item['aspect'])
            pred_aspect_sentiment_set.add((item['aspect'], item['sentiment']))

        pred_aspects.append(pred_aspect_set)
        pred_aspect_sentiments.append(pred_aspect_sentiment_set)

    all_true_aspects = set.union(*true_aspects) if true_aspects else set()
    all_pred_aspects = set.union(*pred_aspects) if pred_aspects else set()

    tp_aspect = sum(len(gt & pred) for gt, pred in zip(true_aspects, pred_aspects))
    fp_aspect = sum(len(pred - gt) for gt, pred in zip(true_aspects, pred_aspects))
    fn_aspect = sum(len(gt - pred) for gt, pred in zip(true_aspects, pred_aspects))

    precision_aspect = tp_aspect / (tp_aspect + fp_aspect + 1e-8)
    recall_aspect = tp_aspect / (tp_aspect + fn_aspect + 1e-8)
    f1_aspect = 2 * precision_aspect * recall_aspect / (precision_aspect + recall_aspect + 1e-8)

    tp_sentiment = sum(len(gt & pred) for gt, pred in zip(true_aspect_sentiments, pred_aspect_sentiments))
    fp_sentiment = sum(len(pred - gt) for gt, pred in zip(true_aspect_sentiments, pred_aspect_sentiments))
    fn_sentiment = sum(len(gt - pred) for gt, pred in zip(true_aspect_sentiments, pred_aspect_sentiments))

    precision_sentiment = tp_sentiment / (tp_sentiment + fp_sentiment + 1e-8)
    recall_sentiment = tp_sentiment / (tp_sentiment + fn_sentiment + 1e-8)
    f1_sentiment = 2 * precision_sentiment * recall_sentiment / (precision_sentiment + recall_sentiment + 1e-8)

    return {
        "Aspect Detection F1": f1_aspect,
        "Sentiment Classification F1": f1_sentiment
    }


In [46]:
client = OpenAI()
predictions = []

# Tạo few-shot messages
few_shot_messages = []
for ex in few_shot_json:
    few_shot_messages.append({
        "role": "user",
        "content": f"Extract aspects and sentiments from the following review:\n{ex['text']}"
    })
    few_shot_messages.append({
        "role": "function",
        "name": "extract_aspect_sentiment",
        "content": json.dumps({"results": ex["sentiments"]})
    })

# Dự đoán
for data in tqdm(test_json):
    messages = [
        {"role": "system", "content": "You are an AI assistant that extracts aspects and their sentiments from text."},
        *few_shot_messages,
        {"role": "user", "content": f"Extract aspects and sentiments from the following review:\n{data['text']}"}
    ]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        functions=gpt_functions,
        function_call={"name": "extract_aspect_sentiment"},
        temperature=0
    )

    output = response.choices[0].message.function_call.arguments
    parsed_output = json.loads(output)
    predictions.append(parsed_output)

100%|██████████| 100/100 [01:23<00:00,  1.20it/s]


In [47]:
scores = evaluate_aspect_sentiment(test_json, predictions)
print(scores)

{'Aspect Detection F1': 0.9126213542055096, 'Sentiment Classification F1': 0.7729468548959136}


In [48]:
result_file = path.join(RESULT_DIR, 'ViABSA_BP_Few-shot.json')
with open(result_file, 'w') as f:
    json.dump(predictions, f, indent=4, ensure_ascii=False)